# 0) install InstagramAPI by running in your terminal: "pip install InstagramAPI"

In [ ]:
from InstagramAPI import InstagramAPI
import pandas as pd
from pandas.io.json import json_normalize
from time import sleep
import requests
import os
import shutil
import time

In [ ]:
# Create target Directory if doesn't exist
Folder_Name = 'images'

try:
    shutil.rmtree(Folder_Name)
    print("Deleted previous folder")
except OSError as e:
    print(f'Error: {Folder_Name} : {e.strerror}')

In [ ]:
if not os.path.exists(Folder_Name):
    os.mkdir(Folder_Name)
    print("Directory " , Folder_Name ,  " Created ")
else:    
    print("Directory " , Folder_Name ,  " already exists")

# 1) LOGIN

In [ ]:
def login_to_instagram(username, password):
    api = InstagramAPI(username, password)
    api.login()
    return api

# CHANGE USERNAME AND PASSWORD BELOW

Becareful with your credentials, I recommend creating another fake instagram account and using these credentials below instead of real ones

In [ ]:
username = ""
password = ""
api = login_to_instagram(username,password) 
#Don't worry if error message first occur as long as it ends with "Login success"

# 2) Various Preprocessing Necessary functions

In [ ]:
def get_largest_image(candidates):
    candidate = {}
    pixels = 0
    for cand in candidates:
        # pick the highest resolution one
        res = cand['height']*cand['width']
        if res > pixels:
            pixels = res
            candidate = cand
    return candidate


In [ ]:
def save_image_from_candidate(url):
    filename = ''
    response = requests.get(url)
    print (response)
    # check the response status code, 200 means good
    if response.status_code == 200: 
        filename = url.split("/")[-1].split('?')[0]
        with open('images/'+ filename, 'wb') as f:
            f.write(response.content)
            print(filename)
    return filename

In [ ]:
def get_their_posts(api, user_id):
    '''Retrieve all posts from someone else's profile'''
    images = []
    has_more_posts = True
    max_id= ''

    while has_more_posts:
        api.getUserFeed(user_id, maxid=max_id)
        if api.LastJson['more_available'] is not True:
            has_more_posts = False #stop condition

        max_id = api.LastJson.get('next_max_id','')
        for item in api.LastJson['items']:
            if item["media_type"] ==1 and 'image_versions2' in item.keys(): #corresponds to images type of content, we don't want videos here
                candidate = get_largest_image(item['image_versions2']['candidates'])
                filename = save_image_from_candidate(candidate['url'])
                if filename != '':
                    try:
                        caption = item['caption']['text']
                    except:
                        caption ="None"
                    try:
                        likes = item['like_count']
                    except:
                        likes = 0
                    try:
                        comments = item['comment_count']
                    except:
                        comments = 0
                    try:
                        location = item['location']["name"]
                    except:
                        location = "Unknown"
                    try:
                        publication_time = item['taken_at']
                    except:
                        publication = "Unknown"
                    images.append((filename, caption,likes,comments,location,publication_time))
                   # sleep(2)
            elif item['media_type'] == 8 : #corresponds to carousels type of content
                for carousel in item['carousel_media']:
                ###We don't care about videos in carousels
                    if carousel['media_type'] == 1:
                        candidate = get_largest_image(carousel['image_versions2']['candidates'])
                        # get image 
                        filename = save_image_from_candidate(candidate['url'])
                        if filename != '':
                            # get status, save as tuple
                            try:
                                caption = item['caption']['text']
                            except:
                                caption ="None"
                            try:
                                likes = item['like_count']
                            except:
                                likes = 0
                            try:
                                comments = item['comment_count']
                            except:
                                comments = 0
                            try:
                                location = item['location']["name"]
                            except:
                                location = "Unknown"
                            try:
                                publication_time = item['taken_at']
                            except:
                                publication = "Unknown"
                            images.append((filename, caption,likes,comments,location,publication_time))
        if has_more_posts:
            print(str(len(images)) + ' posts retrieved so far...')

    print('Total posts retrieved: ' + str(len(images)))
    
    return images

In [ ]:
api.LastJson

# 3) RETRIEVE POSTS FROM BELOW COMPANY

You need to get the id of the account your target
To get it input the username in the following website:
https://thumbtube.com/instagram-user-id-finder#userid

In [ ]:
user_id = 4790789 #change this id

start = time.time()
content_list = get_their_posts(api, user_id)
# run your code
end = time.time()

elapsed = end - start
print(elapsed)
print(len(content_list))
print(elapsed/len(content_list))

# 4) Transfrom JSON to Dataframe

In [ ]:
content_list
content_df = pd.DataFrame(content_list) 
content_df.columns =['name_of_file','caption','likes_count',"comments_count","location","publication_time"]
content_df
content_df.to_csv("images_extracted.csv")

In [ ]:
content_df

# OTHER USEFUL FUNCTIONS

In [ ]:
def get_my_posts(api):
    '''Retrieve all posts from own profile'''
    my_posts = []
    has_more_posts = True
    max_id= ''

    while has_more_posts:
        api.getSelfUserFeed(maxid=max_id)
        if api.LastJson['more_available'] is not True:
            has_more_posts = False #stop condition

        max_id = api.LastJson.get('next_max_id','')
        my_posts.extend(api.LastJson['items']) #merge lists
        sleep(2) # slows down to avoid flooding

        if has_more_posts:
            print(str(len(my_posts)) + ' posts retrieved so far...')

    print('Total posts retrieved: ' + str(len(my_posts)))
    
    return my_posts

my_posts = get_my_posts(api)

In [ ]:
def get_images_from_hashtag(hashtag, num_images):
    images = []
    

    if get_hashtag == False:
        return images

    for item in my_insta_api.LastJson['items']:
        if item['media_type'] == 1 and 'image_versions2' in item.keys():
            candidate = get_largest_image(item['image_versions2']['candidates'])
            # get image 
            filename = save_image_from_candidate(candidate['url'])
            if filename != '':
                # get status, save as tuple
                caption = item['caption']['text']
                images.append((filename, caption))
            if len(images) >= num_images:
                break
    return images

In [ ]:
def get_posts_likers(api, my_posts):
    '''Retrieve all likers on all posts'''
    
    likers = []
    
    print('wait %.1f minutes' % (len(my_posts)*2/60.))
    for i in range(len(my_posts)):
        m_id = my_posts[i]['id']
        api.getMediaLikers(m_id)
        
        likers += [api.LastJson]
        
        # Include post_id in likers dict list
        likers[i]['post_id'] = m_id
        
        sleep(2)
    print('done')
    
    return likers


likers = get_posts_likers(api, my_posts)  

In [ ]:
def get_posts_commenters(api, my_posts):
    '''Retrieve all commenters on all posts '''
    
    commenters = []
    
    print('wait %.1f minutes' % (len(my_posts)*2/60.))
    for i in range(len(my_posts)):
        m_id = my_posts[i]['id']
        api.getMediaComments(m_id)
        
        commenters += [api.LastJson]
        
        # Include post_id in commenters dict list
        commenters[i]['post_id'] = m_id
            
        sleep(2)
    print('done')
    
    return commenters

commenters = get_posts_commenters(api, my_posts)


In [ ]:
def posts_likers_to_df(likers):
    '''Transforms likers list of dicts into pandas DataFrame'''
    
    # Normalize likers by getting the 'users' list and the post_id of each like
    df_likers = json_normalize(likers, 'users', ['post_id'])
    
    # Add 'content_type' column to know the rows are likes
    df_likers['content_type'] = 'like'
    
    return df_likers

def posts_commenters_to_df(commenters):
    '''Transforms commenters list of dicts into pandas DataFrame'''
    
    # Include username and full_name of commenter in 'comments' list of dicts
    for i in range(len(commenters)):
        if len(commenters[i]['comments']) > 0: # checks if there is any comment on the post
            for j in range(len(commenters[i]['comments'])):
                # Puts username/full_name one level up
                commenters[i]['comments'][j]['username'] = commenters[i]['comments'][j]['user']['username']
                commenters[i]['comments'][j]['full_name'] = commenters[i]['comments'][j]['user']['full_name']
                
    # Create DataFrame
    # Normalize commenters to have 1 row per comment, and gets 'post_id' from parent 
    df_commenters = json_normalize(commenters, 'comments', 'post_id')
    
    # Get rid of 'user' column as we already handled it above
    del df_commenters['user']
    
    return df_commenters

df_likers = posts_likers_to_df(likers)
df_commenters = posts_commenters_to_df(commenters)

In [ ]:
print('Total posts: ' + str(len(my_posts)))
print('---------')
print('Total likes on profile: ' + str(df_likers.shape[0])) #shape[0] represents number of rows
print('Distinct users that liked your posts: ' +str(df_likers.username.nunique())) # nunique() will count distinct values of a col
print('---------')
print('Total comments on profile: ' + str(df_comment.shape[0]))
print('Distinct users that commented your posts: ' +str(df_comment.username.nunique()))

In [ ]:
df_likers.username.value_counts()[:10]

In [ ]:
df_likers.username.value_counts()[:10].plot(kind='bar', title='Top 10 media likers', grid=True, figsize=(12,6))

In [ ]:
df_likers.username.value_counts()[:10].plot(kind='pie', title='Top 10 media likers distribution', autopct='%1.1f%%', figsize=(12,6))

In [ ]:
df_commenters['username'].value_counts()[:10].plot(kind='bar', figsize=(12,6), title='Top 10 post commenters')

In [ ]:
# Converts date from unix time to YYYY-MM-DD hh24:mm:ss
df_commenters.created_at = pd.to_datetime(df_commenters.created_at, unit='s')
df_commenters.created_at_utc = pd.to_datetime(df_commenters.created_at_utc, unit='s')

In [ ]:
df_commenters.created_at.dt.weekday.value_counts().sort_index().plot(kind='bar', figsize=(12,6), title='Comments per day of the week (0 - Sunday, 6 - Saturday)')

In [ ]:
df_commenters.created_at.dt.hour.value_counts().sort_index().plot(kind='bar', figsize=(12,6))